# Image Classification Model - Inference Server Function
________________________________________________________

The function accepts a URL or binary image and provides an estimated binary-class predictionusing the tensorflow model developed in **[Deploy a tensorflow-horovod job as a pipeline](mlrun_mpijob_pipe.ipynb)**.

In [1]:
# nuclio: ignore
# !pip install -U tensorflow==1.14.0 numpy==1.16.4 pandas

In [2]:
# nuclio: ignore
import nuclio

### **install dependencies and set config**
**Note**: In this demonstration since we pull tensorflow a Tensorflow 1.14 image directly from Docker Hub, we _**do not need to directly install it as a build command**_.

In [3]:
%nuclio cmd -c pip install -U numpy==1.16.4 azure install keras requests pillow

In [4]:
%nuclio config spec.build.baseImage = "tensorflow/tensorflow:1.14.0-py3"

%nuclio: setting spec.build.baseImage to 'tensorflow/tensorflow:1.14.0-py3'


### **set function environment variables**

In [5]:
%%nuclio env 
IMAGE_WIDTH=128
IMAGE_HEIGHT=128

%nuclio: setting 'IMAGE_WIDTH' environment variable
%nuclio: setting 'IMAGE_HEIGHT' environment variable


### **function code**

In [6]:
import os
import json
import numpy as np
import requests
from tensorflow import keras
from keras.models import load_model
from keras.preprocessing import image
from keras.preprocessing.image import load_img
from os import environ, path
from PIL import Image
from io import BytesIO
from urllib.request import urlopen
from mlrun.execution import MLClientCtx

Using TensorFlow backend.


### **Model Serving Class**

In [7]:
class TFModel(object):
    def __init__(self,
                 name: str, 
                 model_dir: str
    ):
        """Model server
        
        :param name:      name of server
        :param model_dir: destination folder of estimated model
        """
        self.name = name
        self.model_filepath = model_dir
        self.model = None
        self.ready = None

        self.IMAGE_WIDTH = int(environ['IMAGE_WIDTH'])
        self.IMAGE_HEIGHT = int(environ['IMAGE_HEIGHT'])
        
        try:
            print(environ['classes_map'])
            with open(environ['classes_map'], 'r') as f:
                self.classes = json.load(f)
        except:
            self.classes = None
        
        print(f'Classes: {self.classes}')

    def load(self):
        """Load serialized tensorflow model
        """
        self.model = load_model(self.model_filepath)

        self.ready = True

    def _download_file(self, url, target_path):
        with requests.get(url, stream=True) as response:
            response.raise_for_status()
            with open(target_path, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    if chunk:
                        f.write(chunk)

    def predict(
        self,
        context:  MLClientCtx,
        data
    ):
        """Predict given data and estimated model
        
        :param context: MLClientCTx
        :param data:    in this demonstation and byte array representing the image(s) submitted
                        for classification
        """
        img = Image.open(BytesIO(data))
        img = img.resize((self.IMAGE_WIDTH, self.IMAGE_HEIGHT))

        # Load image
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        images = np.vstack([x])

        # Predict
        predicted_probability = self.model.predict(images)

        # return prediction
        if self.classes:
            predicted_classes = np.around(predicted_probability, 1).tolist()[0]
            predicted_probabilities = predicted_probability.tolist()[0]
            #print(predicted_classes)
            #print(predicted_probabilities)
            return {
                'prediction': [self.classes[str(int(cls))] for cls in predicted_classes], 
                f'{self.classes["1"]}-probability': predicted_probabilities
            }
        else:
            return predicted_probability.tolist()[0]

### **routes**

In [8]:
def predict(context, model_name, event):
    global models
    global protocol

    # Load the requested model
    model = models[model_name]

    # Verify model is loaded (Async)
    if not model.ready:
        model.load()
    
    # extract image data from event
    try:
        data = event.body
        ctype = event.content_type
        if not ctype or ctype.startswith('text/plain'):
            # Get image from URL
            url = data.decode('utf-8')
            context.logger.debug_with('downloading image', url=url)
            data = urlopen(url).read()
            
    except Exception as e:
        raise Exception("Failed to get data: {}".format(e))                
            
    # Predict
    results = model.predict(context, data)
    context.logger.info(results)

    # Wrap & return response
    return context.Response(body=json.dumps(results),
                            headers={},
                            content_type='text/plain',
                            status_code=200)

# Router
paths = {
    'predict': predict,
    'explain': '',
    'outlier_detector': '',
    'metrics': '',
}

### main

In [9]:
model_prefix = 'SERVING_MODEL_'
models = {}

def init_context(context):
    global models
    global model_prefix

    # Initialize models from environment variables
    # Using the {model_prefix}_{model_name} = {model_path} syntax
    model_paths = {k[len(model_prefix):]: v for k, v in environ.items() if
                   k.startswith(model_prefix)}

    models = {name: TFModel(name=name, model_dir=path) for name, path in
              model_paths.items()}
    context.logger.info(f'Loaded {list(models.keys())}')

In [10]:
err_string = 'Got path: {}\nPath must be <host>/<action>/<model-name> \nactions: {} \nmodels: {}'

def handler(context, event):
    global models
    global paths

    # check if valid route & model
    sp_path = event.path.strip('/').split('/')
    if len(sp_path) < 2 or sp_path[0] not in paths or sp_path[1] not in models:
        return context.Response(body=err_string.format(event.path, '|'.join(paths), '|'.join(models.keys())),
                                content_type='text/plain',
                                status_code=400)
        
    function_path = sp_path[0] 
    model_name = sp_path[1]

    context.logger.info(
        f'Serving uri: {event.path} for route {function_path} '
        f'with {model_name}, content type: {event.content_type}')

    route = paths.get(function_path)
    if route:
        return route(context, model_name, event)

    return context.Response(body='function {} not implemented'.format(function_path),
                            content_type='text/plain',
                            status_code=400)

In [11]:
# nuclio: end-code

### **test locally**

Make sure your local TF / Keras version is the same as pulled in the nuclio image for accurate testing

Set the served models and their file paths using: `SERVING_MODEL_<name> = <model file path>`

> Note: this notebook assumes the model and categories are under <b>/User/mlrun/examples/</b>

In [12]:
base_dir = os.getcwd()
environ['SERVING_MODEL_cat_dog_v1'] = base_dir + 'models/cats_n_dogs.h5'
environ['classes_map'] = base_dir + 'images/categories_map.json'

init_context(context)

NameError: name 'os' is not defined

In [ ]:
from PIL import Image
from io import BytesIO
import matplotlib.pyplot as plt

cat_image_url = 'https://s3.amazonaws.com/iguazio-sample-data/images/catanddog/cat.102.jpg'
response = requests.get(cat_image_url)
img = Image.open(BytesIO(response.content))
plt.imshow(img)

model_name = 'cat_dog_v1'
event = nuclio.Event(body=response.content,
                     content_type='image/jpeg',
                     path=f'/predict/{model_name}')
output = handler(context, event)
print(str(output.body))

### **deploy the serving function to the cluster**

In [ ]:
# convert the notebook code to deployable function, configure it
from mlrun import code_to_function
fn = code_to_function('tf-image-server-from-notebook', runtime='nuclio')

# set the API/trigger, attach the home dir to the function
fn.with_http(workers=2).add_volume('User','~/')

# set the model file path SERVING_MODEL_<name> = <model file path>
fn.set_env('SERVING_MODEL_cat_dog_v1', base_dir + 'models/cats_n_dogs.h5')
fn.set_env('classes_map', base_dir + 'images/categories_map.json')

In [ ]:
# deploy the function to the cluster
addr = fn.deploy(project='nuclio-serving')

### **test the function (jpeg url)**

In [ ]:
dog_image_url = 'https://s3.amazonaws.com/iguazio-sample-data/images/catanddog/dog.102.jpg'
response = requests.get(dog_image_url)
img = Image.open(BytesIO(response.content))
plt.imshow(img)

headers = {'Content-type': 'text/plain'}
response = requests.post(url=addr + f'/predict/{model_name}', data=dog_image_url, headers=headers)
print(response.content.decode('utf-8'))

### **test the function (jpeg image)**

In [ ]:
cat_image_url = 'https://s3.amazonaws.com/iguazio-sample-data/images/catanddog/cat.102.jpg'
response = requests.get(cat_image_url)
img = Image.open(BytesIO(response.content))
plt.imshow(img)

headers = {'Content-type': 'image/jpeg'}
response = requests.post(url=addr + f'/predict/{model_name}', data=response.content, headers=headers)
print(response.content.decode('utf-8'))